# Helpers

In [2]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from scipy.interpolate import griddata

In [3]:
csv_dir = "../output/"

In [97]:
def parse_boundary(csv_path):
    data = pd.read_csv(csv_path, header=None)
    x, y, z = data[0], data[1], data[2]
    # x, y = (x + 1) / 2, (y + 1) / 2
    return x, y, z

In [205]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from scipy.interpolate import griddata

def parse_csv_for_xy(path):
    # Read CSV assuming pairs of x, y coordinates (x1, y1, x2, y2, ...)
    data = pd.read_csv(path, header=None)
    
    # Convert all values to numeric, forcing errors to NaN (which makes them easier to handle)
    data = data.apply(pd.to_numeric, errors='coerce')

    # Flatten the data into x and y arrays
    x_vals = data.iloc[:, 0::2].values.flatten()  # Extract x values (every other column starting from 0)
    y_vals = data.iloc[:, 1::2].values.flatten()  # Extract y values (every other column starting from 1)

    return x_vals, y_vals

def get_original_points(s = 16):
    x = []
    y = []
    for j in range(s):
        for i in range(s):
            x.append((i / (s - 1)) * 2 - 1)
            y.append((j / (s - 1)) * 2 - 1)
    x = np.array(x)
    y = np.array(y)
    x = np.clip(x, 0, 1)
    y = np.clip(y, 0, 1)
    return x, y
        

def visualize_csv_output_vector(path, output_path="./solver_output_plot.png", original_point=None, xstart=0, ystart=0, xend=1, yend=1, surface=False):
    # Parse CSV to get x, y values
    x_vals, y_vals = parse_csv_for_xy(path)
    
    # Remove NaN values from the data
    valid_mask = ~np.isnan(x_vals) & ~np.isnan(y_vals)  # Create mask for non-NaN values in x and y
    x_vals = x_vals[valid_mask]  
    y_vals = y_vals[valid_mask] 
    
    assert len(x_vals) == len(y_vals), "x_vals and y_vals must have the same length"
    z_vals = np.zeros_like(x_vals)
    
    x = np.linspace(xstart, xend, 100)  # Increase resolution if necessary
    y = np.linspace(ystart, yend, 100)  # Increase resolution if necessary
    X, Y = np.meshgrid(x, y)
    
    # Interpolate the Z values onto the grid (if you want smooth surface interpolation)
    Z_interp = griddata((x_vals, y_vals), z_vals, (X, Y), method='linear')

    # Create a figure for plotting
    fig = go.Figure()

    # Add surface plot with interpolated Z values (if surface is true)
    if surface: 
        fig.add_trace(go.Surface(z=Z_interp, x=X, y=Y, colorscale='Viridis', showscale=True))

    # Add a scatter plot for the original points (if surface is false)
    if not surface:
        fig.add_trace(go.Scatter3d(x=x_vals, y=y_vals, z=z_vals, mode='markers',
            marker=dict(size=2, color='blue') 
        ))
        
    
    # Add boundary if provided
    if original_point is not None: 
        x_original, y_original = get_original_points()
        z_original = np.zeros_like(x_original) - 0.001
        fig.add_trace(go.Scatter3d(x=x_original, y=y_original, z=z_original, mode='markers',
            marker=dict(size=2, color='red')  
        ))

    # Update layout for better visualization
    fig.update_layout(scene=dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Z',
        zaxis=dict(
            range=[-0.01, 0.01]  # Set the Z-axis range (adjust these values as needed)
        ),
        aspectmode='cube'  # Ensures that all axes are scaled equally
    ))

    # Show the plot and save to file
    fig.show()
    fig.write_image(output_path)
    print(f"Figure saved as {output_path}")

In [67]:
import plotly.graph_objects as go
def print_boundary_dirichlet(path=None, shape=None, dir=None): 
    if path is None: path = dir + shape + 'BoundaryDirichlet.csv'
    x, y, z = parse_boundary(path)
    fig = go.Figure()
    for i in range(len(x)):
        fig.add_trace(go.Scatter3d(
            x=[x[i], x[(i + 1) % len(x)]],
            y=[y[i], y[(i + 1) % len(y)]],
            z=[z[i], z[(i + 1) % len(z)]],
            mode='lines',
            line=dict(color='blue', width=2)
        ))
    fig.add_trace(go.Scatter3d(x=x,y=y,z=z,mode='markers',
        marker=dict(size=5, color='red', opacity=0.8)  # Color points
    ))

    fig.update_layout(title_text='3D Star Shape on a Tilted Plane',
                  scene=dict(
                      xaxis_title='X-axis',
                      yaxis_title='Y-axis',
                      zaxis_title='Z-axis'),
                  )

    fig.show()

In [26]:
def get_shape_csv(shape): 
    return shape + ".csv"
def get_boundaryD_csv(shape): 
    return shape + "BoundaryDirichlet.csv"

# Plot Deformed Rectangle

In [202]:
shape = "rectangle_displacement"
shape_csv = get_shape_csv(shape)
boundary_csv = get_boundaryD_csv(shape)

In [203]:
print_boundary_dirichlet(path=csv_dir + boundary_csv) 

In [206]:
visualize_csv_output_vector(csv_dir + shape_csv, original_point=True) 

Figure saved as ./solver_output_plot.png


In [209]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from scipy.interpolate import griddata

def parse_csv_for_xy(path):
    # Read CSV assuming pairs of x, y coordinates (x1, y1, x2, y2, ...)
    data = pd.read_csv(path, header=None)
    
    # Convert all values to numeric, forcing errors to NaN (which makes them easier to handle)
    data = data.apply(pd.to_numeric, errors='coerce')

    # Flatten the data into x and y arrays
    x_vals = data.iloc[:, 0::2].values.flatten()  # Extract x values (every other column starting from 0)
    y_vals = data.iloc[:, 1::2].values.flatten()  # Extract y values (every other column starting from 1)

    return x_vals, y_vals

def get_original_points(s=16):
    x = []
    y = []
    for j in range(s):
        for i in range(s):
            x.append((i / (s - 1)) * 2 - 1)
            y.append((j / (s - 1)) * 2 - 1)
    x = np.array(x)
    y = np.array(y)
    x = np.clip(x, 0, 1)
    y = np.clip(y, 0, 1)
    return x, y

def visualize_displacement_with_arrows(path, output_path="./displacement_vector_field.png", s=16):
    # Parse CSV to get x, y values after deformation
    x_vals_deformed, y_vals_deformed = parse_csv_for_xy(path)
    
    # Get original points in the elastic material
    x_original, y_original = get_original_points(s)
    
    # Ensure lengths match between original and deformed points
    assert len(x_original) == len(x_vals_deformed), "Original and deformed points must have the same length."
    assert len(y_original) == len(y_vals_deformed), "Original and deformed points must have the same length."

    # Create a figure for plotting
    fig = go.Figure()

    # Add a vector field plot to represent displacement
    for x0, y0, x1, y1 in zip(x_original, y_original, x_vals_deformed, y_vals_deformed):
        fig.add_trace(go.Scatter3d(
            x=[x0, x1],  # Line starts at original point and ends at deformed point
            y=[y0, y1],
            z=[0, 0],  # Assuming displacement in 2D plane, set Z to 0
            mode='lines+markers',
            marker=dict(size=2, color='blue'),
            line=dict(color='green', width=2)
        ))
    
    # Add layout adjustments
    fig.update_layout(scene=dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Z',
        aspectmode='cube'
    ))

    # Show the plot and save to file
    fig.show()
    fig.write_image(output_path)
    print(f"Figure saved as {output_path}")


In [210]:
visualize_displacement_with_arrows(csv_dir + shape_csv, output_path=csv_dir + "displacement_vector_field.png")

Figure saved as ../output/displacement_vector_field.png


In [211]:
def visualize_deformed_grid(path, output_path="./deformed_grid.png", s=16):
    # Parse CSV to get x, y values after deformation
    x_vals_deformed, y_vals_deformed = parse_csv_for_xy(path)
    
    # Get original points in the elastic material
    x_original, y_original = get_original_points(s)
    
    # Create a figure for plotting
    fig = go.Figure()

    # Plot the original grid points
    fig.add_trace(go.Scatter3d(
        x=x_original,
        y=y_original,
        z=np.zeros_like(x_original),
        mode='markers',
        marker=dict(size=3, color='red', symbol='circle'),
        name='Original Grid'
    ))

    # Plot the deformed grid points
    fig.add_trace(go.Scatter3d(
        x=x_vals_deformed,
        y=y_vals_deformed,
        z=np.zeros_like(x_vals_deformed),
        mode='markers',
        marker=dict(size=3, color='blue', symbol='circle'),
        name='Deformed Grid'
    ))

    # Update layout for better visualization
    fig.update_layout(scene=dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Z',
        aspectmode='cube'
    ))

    # Show the plot and save to file
    fig.show()
    fig.write_image(output_path)
    print(f"Figure saved as {output_path}")


In [212]:
visualize_deformed_grid(csv_dir + shape_csv, output_path=csv_dir + "deformed_grid.png")

Figure saved as ../output/deformed_grid.png


In [213]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from matplotlib import cm

def parse_csv_for_xy(path):
    # Read CSV assuming pairs of x, y coordinates (x1, y1, x2, y2, ...)
    data = pd.read_csv(path, header=None)
    
    # Convert all values to numeric, forcing errors to NaN (which makes them easier to handle)
    data = data.apply(pd.to_numeric, errors='coerce')

    # Flatten the data into x and y arrays
    x_vals = data.iloc[:, 0::2].values.flatten()  # Extract x values (every other column starting from 0)
    y_vals = data.iloc[:, 1::2].values.flatten()  # Extract y values (every other column starting from 1)

    return x_vals, y_vals

def get_original_points(s=16):
    x = []
    y = []
    for j in range(s):
        for i in range(s):
            x.append((i / (s - 1)) * 2 - 1)
            y.append((j / (s - 1)) * 2 - 1)
    x = np.array(x)
    y = np.array(y)
    x = np.clip(x, 0, 1)
    y = np.clip(y, 0, 1)
    return x, y

def visualize_displacement_with_arrows(path, output_path="./displacement_vector_field.png", s=16):
    # Parse CSV to get x, y values after deformation
    x_vals_deformed, y_vals_deformed = parse_csv_for_xy(path)
    
    # Get original points in the elastic material
    x_original, y_original = get_original_points(s)
    
    # Ensure lengths match between original and deformed points
    assert len(x_original) == len(x_vals_deformed), "Original and deformed points must have the same length."
    assert len(y_original) == len(y_vals_deformed), "Original and deformed points must have the same length."

    # Number of unique colors needed, based on columns
    num_colors = s
    colors = cm.rainbow(np.linspace(0, 1, num_colors))  # Use a colormap to generate unique colors

    # Create a figure for plotting
    fig = go.Figure()

    # Add a vector field plot to represent displacement, color-coded by column
    for idx, (x0, y0, x1, y1) in enumerate(zip(x_original, y_original, x_vals_deformed, y_vals_deformed)):
        col_index = idx % s  # Determine the column index (0 to s-1)
        color = 'rgb({}, {}, {})'.format(*[int(255 * c) for c in colors[col_index][:3]])  # Convert color to RGB
        
        # Add an arrow from the original point to the deformed point
        fig.add_trace(go.Scatter3d(
            x=[x0, x1],  # Line starts at original point and ends at deformed point
            y=[y0, y1],
            z=[0, 0],  # Assuming displacement in 2D plane, set Z to 0
            mode='lines+markers',
            marker=dict(size=2, color=color),
            line=dict(color=color, width=2)
        ))
    
    # Add layout adjustments
    fig.update_layout(scene=dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Z',
        aspectmode='cube'
    ))

    # Show the plot and save to file
    fig.show()
    fig.write_image(output_path)
    print(f"Figure saved as {output_path}")

# Example usage
visualize_displacement_with_arrows(csv_dir + shape_csv, output_path=csv_dir + "displacement_vector_field.png")


Figure saved as ../output/displacement_vector_field.png


### Stress Visualizer 

In [ ]:
import autograd.numpy as np
from autograd import jacobian

def calculate_individual_displacement_gradient_2D(original_points, deformed_points):
    assert len(original_points) == len(deformed_points)
    displacements = deformed_points - original_points
    dx = original_points[1, 0] - original_points[0, 0]
    dy = original_points[1, 1] - original_points[0, 1]
    if dx == 0 or dy == 0:
        raise ValueError("Finite difference requires non-zero changes in x and y.")
    dudx = (displacements[1, 0] - displacements[0, 0]) / dx  # ∂u_x/∂x
    dudy = (displacements[1, 0] - displacements[0, 0]) / dy  # ∂u_x/∂y
    dvdx = (displacements[1, 1] - displacements[0, 1]) / dx  # ∂u_y/∂x
    dvdy = (displacements[1, 1] - displacements[0, 1]) / dy  # ∂u_y/∂y
    gradient = np.array([[dudx, dudy],
                         [dvdx, dvdy]])
    return gradient

def calculate_displacement_gradient_2D(original_points, deformed_points):
    assert len(original_points) == len(deformed_points)
    gradients = np.zeros((len(original_points) - 1, 2, 2))
    for i in range(len(original_points) - 1):
        gradients[i] = calculate_individual_displacement_gradient_2D(original_points[i:i+2], deformed_points[i:i+2])
    return gradients

def calculate_individual_strain_2D(displacement_gradient):
    return 0.5 * (displacement_gradient + displacement_gradient.T)

def calculate_strain_2D(original_points, deformed_points):
    assert len(original_points) == len(deformed_points)
    strain = np.zeros((len(original_points) - 1, 2, 2))
    for i in range(len(original_points) - 1):
        strain[i] = calculate_individual_strain_2D(calculate_individual_displacement_gradient_2D(original_points[i:i+2], deformed_points[i:i+2]))
    return strain 

## linear elasticity 
## 2D isotropic materia
def calculate_individual_stress_2D(strain, lam, mu):
    return lam * np.trace(strain) * np.eye(strain.shape[0]) + 2 * mu * strain

def calculate_stress_2D(original_points, deformed_points, lam, mu):
    assert len(original_points) == len(deformed_points)
    stress = np.zeros((len(original_points) - 1, 2, 2))
    for i in range(len(original_points) - 1):
        displacement_gradient = calculate_individual_displacement_gradient_2D(original_points[i:i+2], deformed_points[i:i+2])
        strain = calculate_individual_strain_2D(displacement_gradient)
        stress[i] = calculate_individual_stress_2D(strain, lam, mu)
    return stress
    
def calculate_traction(stress, normal):
    return np.dot(stress, normal)

In [220]:
deformed_x, deformed_y = parse_csv_for_xy(csv_dir + shape_csv)
original_x, original_y = get_original_points(s = 16)
deformed_points = np.dstack((deformed_x, deformed_y))
original_points = np.dstack((original_x, original_y))
displacement_gradient = calculate_displacement_gradient_2D(original_points, deformed_points)
strain = calculate_strain_2D(original_points, deformed_points)
lam, mu = 0.1, 0.5
stress = calculate_stress_2D(strain, lam, mu)
# traction = calculate_traction(stress, normal)

ValueError: operands could not be broadcast together with shapes (2,) (0,0) 